# Gemini 텍스트 생성

* 모델 종류: https://ai.google.dev/gemini-api/docs/models/gemini?hl=ko&_gl=1

## 텍스트 전용 입력에서 텍스트 생성

In [6]:
from google import genai

client = genai.Client(api_key="GEMINI_API_KEY")

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["How does AI work?"])
print(response.text)

ImportError: cannot import name 'genai' from 'google' (unknown location)

## 텍스트 및 이미지 입력에서 텍스트 생성

* Gemini API는 텍스트와 미디어 파일을 결합한 멀티모달 입력을 지원합니다. 다음 예는 텍스트 및 이미지 입력에서 텍스트를 생성하는 방법을 보여줍니다.

In [ ]:
from PIL import Image
from google import genai

client = genai.Client(api_key="GEMINI_API_KEY")

image = Image.open("/path/to/organ.png")
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[image, "Tell me about this instrument"])
print(response.text)

## 텍스트 스트림 생성

* 기본적으로 모델은 전체 텍스트 생성 프로세스를 완료한 후에 응답을 반환합니다. 전체 결과를 기다리지 않고 대신 스트리밍을 사용하여 부분 결과를 처리하면 더 빠른 상호작용을 실행할 수 있습니다.
* 다음 예에서는 streamGenerateContent 메서드를 사용하여 스트리밍을 구현하여 텍스트 전용 입력 프롬프트에서 텍스트를 생성하는 방법을 보여줍니다.

In [ ]:
from google import genai

client = genai.Client(api_key="GEMINI_API_KEY")

response = client.models.generate_content_stream(
    model="gemini-2.0-flash",
    contents=["Explain how AI works"])
for chunk in response:
    print(chunk.text, end="")

## 채팅 대화 만들기

* Gemini SDK를 사용하면 여러 번의 질문과 응답을 수집할 수 있으므로 사용자가 점진적으로 답변을 찾거나 여러 부분으로 구성된 문제와 관련하여 도움을 받을 수 있습니다. 이 SDK 기능은 대화 기록을 추적하는 인터페이스를 제공하지만, 백그라운드에서는 동일한 generateContent 메서드를 사용하여 응답을 만듭니다.

* 다음 코드 예는 기본 채팅 구현을 보여줍니다.

In [ ]:
from google import genai

client = genai.Client(api_key="GEMINI_API_KEY")

chat = client.chats.create(model="gemini-2.0-flash")
response = chat.send_message("I have 2 dogs in my house.")
print(response.text)
response = chat.send_message("How many paws are in my house?")
print(response.text)
for message in chat._curated_history:
    print(f'role - ', message.role, end=": ")
    print(message.parts[0].text)

* 다음 예와 같이 채팅과 함께 스트리밍을 사용할 수도 있습니다.

In [ ]:
from google import genai

client = genai.Client(api_key="GEMINI_API_KEY")

chat = client.chats.create(model="gemini-2.0-flash")
response = chat.send_message_stream("I have 2 dogs in my house.")
for chunk in response:
    print(chunk.text, end="")
response = chat.send_message_stream("How many paws are in my house?")
for chunk in response:
    print(chunk.text, end="")
for message in chat._curated_history:
    print(f'role - ', message.role, end=": ")
    print(message.parts[0].text)

## 텍스트 생성 구성

* 모델에 전송하는 모든 프롬프트에는 모델이 응답을 생성하는 방식을 제어하는 매개변수가 포함됩니다. GenerationConfig를 사용하여 이러한 매개변수를 구성할 수 있습니다. 매개변수를 구성하지 않으면 모델은 기본 옵션을 사용하며 이는 모델에 따라 다를 수 있습니다.

* 다음 예는 사용 가능한 여러 옵션을 구성하는 방법을 보여줍니다.

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key="GEMINI_API_KEY")

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["Explain how AI works"],
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0.1
    )
)
print(response.text)

## 시스템 안내 추가

* 시스템 안내를 사용하면 특정 요구사항 및 사용 사례에 따라 모델의 동작을 조정할 수 있습니다.

* 모델에 시스템 안내를 제공하면 작업을 이해하고, 보다 맞춤설정된 대답을 생성하고, 모델과의 전체 사용자 상호작용에 대한 특정 가이드라인을 준수하기 위한 추가 컨텍스트를 모델에 제공할 수 있습니다. 최종 사용자가 제공하는 프롬프트와 별도로 시스템 안내를 설정하여 제품 수준 동작을 지정할 수도 있습니다.

* 모델을 초기화할 때 시스템 안내를 설정할 수 있습니다.

In [ ]:
sys_instruct="You are a cat. Your name is Neko."
client = genai.Client(api_key="GEMINI_API_KEY")

response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=sys_instruct),
    contents=["your prompt here"]
)

## 소설 장르, 제목 입력

In [ ]:
import os
from dotenv import load_dotenv

# 환경변수 로드 및 GEMINI_API_KEY 가져오기
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [ ]:
genre = input("소설 장르를 입력해주세요:")
title = input("소설 제목을 입력해주세요:")

print(f"장르: {genre}")
print(f"제목: {title}")

장르: 판타지, 액션, 일상물
제목: 괴식식당


## 소설 세계관 Test

In [9]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

# 환경변수 로드 및 GEMINI_API_KEY 가져오기
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

instruction = """
당신은 전문적으로 소설 세계관을 만드는 작가입니다.
주어진 장르, 제목을 기반으로 독창적이고 생동감 있는 소설 세계관을 만들어주세요.
소설 세계관을 구성할 때, 아래의 항목들을 참고하여 상세하게 작성하세요. 각 항목별 작성 가이드라인을 참고하여 내용을 구성하면 더욱 풍부하고 체계적인 세계관을 만들 수 있습니다.

1. 기본 설정
    - 시대: (예: 과거, 현대, 미래 등) - 시대적 배경과 분위기를 구체적으로 설명해주세요.
    - 핵심 테마: (예: 인간과 자연의 대립, 기술과 마법의 공존 등) - 세계관을 관통하는 핵심 주제를 명확히 제시하고, 다른 요소들과의 연관성을 설명해주세요.

2. 지리적 배경
    - 세계 전체의 구조(대륙, 국가, 도시, 마을 등) - 지리적 구조를 시각적으로 표현하고, 각 지역의 특징을 설명해주세요.
    - 주요 지형 및 자연 환경(기후, 산맥, 강 등) - 자연 환경이 세계관에 미치는 영향을 설명하고, 상징적인 의미를 부여할 수 있습니다.
    - 상징적이거나 중요한 장소(예: 전설적인 성, 신비로운 숲 등) - 각 장소의 역사, 특징, 중요성 등을 구체적으로 설명해주세요.

3. 역사와 문화
    - 세계의 건국 신화 및 주요 역사적 사건 - 역사적 사건이 세계관에 미친 영향과 현재 사회에 남은 유산을 설명해주세요.
    - 사회 구조(정치 체제, 경제 시스템, 계층 구조 등) - 사회 구조가 세계관 구성원들의 삶에 미치는 영향을 설명해주세요.
    - 문화와 종교(전통, 의식, 종교적 신념 등) - 문화와 종교가 세계관에 미치는 영향을 설명하고, 독특한 요소들을 강조해주세요.

4. 기술 및 초자연적 요소
    - 세계 내 기술 수준(과거, 현대, 미래 기술) - 기술 수준이 세계관에 미치는 영향과 특징을 설명해주세요.
    - 만약 존재한다면 마법이나 초자연적 현상의 규칙과 한계 - 마법/초자연 현상이 세계관에 미치는 영향과 작동 방식을 구체적으로 설명해주세요.

5. 인물 및 종족
    - 주요 인물이나 집단, 종족의 특징과 역할 - 각 인물/종족의 개성과 세계관 내 역할을 명확하게 설명해주세요.
    - 각 인물/종족이 속한 사회적, 문화적 배경 - 인물/종족의 배경이 그들의 행동 양식에 미치는 영향을 설명해주세요.

6. 갈등 구조
    - 사회 내부 혹은 외부의 갈등 요소(정치적 분쟁, 종족 간 충돌 등) - 갈등의 원인, 진행 과정, 결과를 구체적으로 설명해주세요.
    - 이러한 갈등이 전체 세계관에 미치는 영향 - 갈등이 세계관의 다른 요소들에 미치는 영향을 설명해주세요.

7. 세부 설정 및 고유 용어
    - 세계관 내에서만 사용되는 독특한 용어나 법칙 정리 - 용어의 의미와 사용 예시를 설명하여 세계관의 현실성을 높여주세요.
    - 설정의 일관성을 유지할 수 있도록 참고 자료 작성 - 필요한 경우, 추가적인 자료를 작성하여 세계관의 완성도를 높여주세요.

이 정보를 바탕으로, 소설 세계관에 대해 상세한 설명을 작성해 주세요.
"""

model = genai.GenerativeModel(
    "models/gemini-2.0-flash", 
    system_instruction=instruction
)

prompt = f"""
## 소설 장르: {genre}
## 소설 제목: {title}

**소설 세계관**
"""

response = model.generate_content(prompt)
worldview = response.text
print(worldview)

## 소설 세계관: 괴식식당

**1. 기본 설정**

*   **시대:** 현대와 유사한 시대, 단 마나와 이종족이 존재하는 평행 세계
*   **핵심 테마:** 괴식(怪食)을 통해 인간과 이종족 간의 소통과 화합, 그리고 일상 속 작은 행복의 발견. 겉으로는 기괴해 보이지만, 그 안에 담긴 이야기와 정을 통해 서로를 이해하고 성장하는 과정을 그린다. '다름'을 인정하고 존중하는 메시지를 괴식을 매개로 전달한다.

**2. 지리적 배경**

*   **세계 전체 구조:**
    *   **인간 거주 지역:** 현대 사회와 유사한 도시 구조를 가지고 있으며, 기술 발전 수준도 비슷하다. 다만, 마나 농도가 높은 지역은 이종족과 공존하는 특별 구역으로 지정되어 있다.
    *   **이종족 거주 지역:** 자연 친화적인 환경을 선호하며, 인간과의 교류가 드문 폐쇄적인 마을이나 고대 유적지에 거주하는 경우가 많다. 숲, 산, 강 등 자연환경과 조화를 이루는 독특한 건축 양식을 보인다.
    *   **중립 지역:** 인간과 이종족이 함께 거주하며 교류하는 지역. 대표적으로 '괴식거리'가 존재하며, 다양한 이종족의 식당과 상점이 즐비하다.
*   **주요 지형 및 자연 환경:**
    *   **마나 농도:** 지역마다 마나 농도가 다르며, 이는 이종족의 거주 분포와 생태계에 큰 영향을 미친다. 마나 농도가 높은 지역은 특이한 식물과 동물이 서식하며, 마법 사용이 용이하다.
    *   **기후:** 대체로 온난한 기후를 유지하나, 마나의 영향으로 인해 예측 불가능한 기상 현상이 발생하기도 한다. 특정 지역에서는 계절이 뒤바뀌거나, 하루에도 여러 번 날씨가 변하는 등 기이한 현상이 나타난다.
    *   **괴수의 숲:** 깊은 숲 속에는 마나의 영향을 받아 변이된 괴수들이 서식한다. 이들은 인간과 이종족에게 위협적인 존재이지만, 동시에 희귀한 식재료를 제공하기도 한다.
*   **상징적이거나 중요한 장소:**
    *   **괴식거리:** 인간과 이종

## 소설 줄거리 Test

In [10]:
import os
from dotenv import load_dotenv

# 환경변수 로드 및 GEMINI_API_KEY 가져오기
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)

instruction = """
당신은 전문적으로 소설 줄거리를 만드는 작가입니다.
주어진 장르, 제목, 세계관을 기반으로 독창적이고 생동감 있는 소설 줄거리를 만들어주세요.
소설 줄거리를 구성할 때, 아래의 항목들을 참고하세요.

1. 기본 정보
   - 장르: [예: 판타지, SF 등]
   - 제목: [제목 입력]
   - 세계관: [세계관의 배경, 시대, 문화, 기술 등 간단히 설명]

2. 도입부
   - 소설이 펼쳐지는 세계관 소개
   - 주요 인물 및 그들의 기본 목표
   - 초기 갈등이나 문제의 암시

3. 전개부
   - 주된 갈등 및 도전 과제 전개
   - 인물 간의 관계와 서브 플롯 설명
   - 주인공이 맞서야 하는 문제와 성장 과정

4. 클라이맥스
   - 갈등의 최고조와 결정적 순간
   - 주인공의 중요한 선택 및 대립의 절정

5. 결말
   - 갈등 해결 및 인물 변화
   - 세계관에 미친 영향과 여운 남기기

위 템플릿에 따라 소설의 줄거리를 상세하게 작성해 주세요. 
"""

model = genai.GenerativeModel(
    "models/gemini-2.0-flash", 
    system_instruction=instruction
)

prompt = f"""
## 소설 장르: {genre}
## 소설 제목: {title}
## 소설 세계관: {worldview}

**소설 줄거리**
"""

response = model.generate_content(prompt)
synopsis = response.text
print(synopsis)

## 소설 줄거리: 괴식식당

**1. 기본 정보**

*   **장르:** 판타지, 액션, 일상물
*   **제목:** 괴식식당
*   **세계관:** 마나와 이종족이 존재하는 현대와 유사한 시대. 괴식을 통해 인간과 이종족 간의 소통과 화합을 추구한다.

**2. 도입부**

*   **세계관 소개:**
    *   마나와 이종족이 존재하는 세계, 인간과 이종족은 오랜 전쟁 끝에 공존을 모색하고 있지만, 여전히 불신과 갈등이 남아있다.
    *   '괴식거리'는 인간과 이종족이 함께 운영하는 식당과 상점이 밀집된 특별 구역으로, 다양한 괴식을 맛볼 수 있으며 문화 교류가 활발하다.
*   **주요 인물 및 기본 목표:**
    *   **강태준:** 괴식 식당 '만월식당'의 젊은 인간 요리사. 뛰어난 요리 실력과 긍정적인 성격을 가졌지만, 괴식에 대한 편견과 싸워야 한다. 목표는 만월식당을 괴식거리 최고의 맛집으로 만들고, 괴식을 통해 인간과 이종족 간의 벽을 허무는 것이다.
    *   **세이렌:** 아름다운 외모를 가진 세이렌 종족의 여성. 뛰어난 노래 실력과 신비로운 분위기를 가지고 있다. 인간에 대한 불신이 강하지만, 태준의 진심에 조금씩 마음을 열어간다. 만월식당의 홀서빙을 담당하며 태준을 돕는다.
*   **초기 갈등 암시:**
    *   만월식당은 괴식거리에서도 가장 인기가 없는 식당 중 하나. 손님은 거의 없고, 재료 수급도 어렵다.
    *   태준은 새로운 메뉴 개발과 식당 홍보를 위해 고군분투하지만, 쉽지 않다.
    *   괴식에 대한 편견을 가진 손님들의 악평과 이종족들의 경계심은 태준을 더욱 힘들게 한다.
    *   만월식당 주변에서 수상한 사건들이 발생하기 시작하고, 태준은 이 사건들이 만월식당과 관련 있음을 직감한다.

**3. 전개부**

*   **주된 갈등 및 도전 과제 전개:**
    *   태준은 만월식당을 살리기 위해 새로운 괴식 메뉴 개발에 박차를 가한다. 이종족들의 입맛을 사로잡기 위해 다양한

## 소설 등장인물 TEST

In [15]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)

instruction = """
당신은 전문적으로 등장인물을 구성하는 소설 작가입니다. 
주어진 장르, 제목, 세계관, 줄거리를 기반으로 소설 등장인물을 만들어주세요.

각 등장인물은 다음 속성을 포함하는 JSON 형태(dict)로 표현해야 합니다.

* 이름: (예: 홍길동, 춘향이 등) - 등장인물의 이름 (필수)
* 성별: (예: 남, 여, 기타) - 등장인물의 성별 (필수)
* 나이: (예: 20세, 30대 초반 등) - 등장인물의 나이 (필수)
* 역할: (예: 주인공, 조력자, 악당 등) - 이야기 속 역할 (필수)
* 직업: (예: 의사, 학생, 무사 등) - 등장인물의 직업 (필수)
* 프로필:
    - (예: 키 180cm, 날카로운 눈매, 과묵한 성격 등) - 외모, 성격, 능력 등 세부 묘사 (선택)
    - (예: 특정 능력, 습관, 버릇, 가치관 등) - 등장인물의 개성을 드러내는 특징 (선택)
    - (예: 가문, 출신, 과거 등) - 등장인물의 과거와 배경 (선택)
    - (예: 주인공과 친구, 연인 관계 등) - 다른 등장인물과의 관계 (선택)

여러 명의 등장인물을 생성할 수 있으며, 각 등장인물은 아래와 같은 형태로 표현해야 합니다.

characters = [
    {
        "이름": "홍길동",
        "성별": "남",
        "나이": "20",
        "역할": "주인공",
        "직업": "무사",
        "프로필": "활달한 성격",
        "특징": "뛰어난 검술"
    }
]"""

model = genai.GenerativeModel(
    "models/gemini-2.0-flash", 
    system_instruction=instruction
)

characters = ""

prompt = f"""
## 소설 장르: {genre}
## 소설 제목: {title}
## 소설 세계관: {worldview}
## 소설 줄거리: {synopsis}
## 기존 소설 등장인물: {characters}

**새로운 소설 등장인물**
"""

response = model.generate_content(prompt)
new_characters = response.text
characters += new_characters
print(characters)

```json
[
    {
        "이름": "강태준",
        "성별": "남",
        "나이": "20대 후반",
        "역할": "주인공",
        "직업": "만월식당 요리사",
        "프로필": {
            "외모": "다부진 체격, 헝클어진 머리, 앞치마가 잘 어울리는 호감형 인상",
            "성격": "낙천적이고 긍정적, 요리에 대한 열정이 넘침, 사람들과 쉽게 친해지는 붙임성",
            "특징": "뛰어난 요리 실력, 새로운 괴식 레시피 개발에 능함, 어려운 상황에서도 웃음을 잃지 않음",
            "과거": "어릴 적 할머니에게 요리를 배움, 할머니의 만월식당을 물려받아 운영 중",
            "관계": "세이렌과 연인 관계, 괴식거리 다른 식당 주인들과 좋은 관계 유지"
        }
    },
    {
        "이름": "세이렌",
        "성별": "여",
        "나이": "불명 (외형은 20대 초반)",
        "역할": "주인공 조력자, 서브 주인공",
        "직업": "만월식당 홀서빙",
        "프로필": {
            "외모": "신비로운 분위기의 미녀, 푸른 머리카락과 깊은 바다색 눈동자, 아름다운 목소리",
            "성격": "처음에는 차갑고 경계심이 강하지만, 점차 마음을 열면 따뜻하고 순수한 모습",
            "특징": "세이렌 종족 특유의 아름다운 노래 실력, 물을 다루는 능력, 인간에 대한 불신",
            "과거": "인간과의 전쟁으로 가족을 잃음, 인간을 증오하며 숨어 살았음",
            "관계": "강태준과 연인 관계, 만월식당 손님들에게 친절하게 대함"
        }
    },
    {
        "이름": "고블린 장군",
        "성별": "남",
        "

## 소설 초안 TEST

In [17]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)

instruction = """당신은 창의적이고 독창적인 소설 작가입니다. 
주어진 장르, 제목, 세계관, 줄거리, 등장인물을 기반으로 소설의 초안을 작성해야 합니다.
장르의 분위기에 맞게 전개하되, 500-1000자 정도의 내용을 작성해야 합니다."""

model = genai.GenerativeModel(
    "models/gemini-2.0-flash", 
    system_instruction=instruction
)

prompt = f"""
## 소설 장르: {genre}
## 소설 제목: {title}
## 소설 세계관: {worldview}
## 소설 줄거리: {synopsis}
## 소설 등장인물: {characters}

**소설 초안**
"""

response = model.generate_content(prompt)
first_chapter = response.text
print(first_chapter)

어스름한 새벽, 만월(滿月)식당의 문이 열렸다. 낡은 나무 간판 아래, ‘괴식 전문’이라 적힌 글자가 희미하게 빛났다. 주방에서는 다부진 체격의 청년, 강태준이 분주하게 움직이고 있었다. 헝클어진 머리칼 아래, 앞치마를 두른 그의 얼굴에는 피곤함과 함께 설렘이 어려 있었다.

"오늘의 괴식은 뭘로 할까나…"

태준은 혼잣말처럼 중얼거리며 냉장고 문을 열었다. 꿈틀거리는 지렁이 젤리, 끈적한 점액질이 가득한 슬라임 푸딩, 징그러운 형상의 괴수 육포 등, 평범한 사람이라면 질색할 만한 재료들이 가득했다. 하지만 태준의 눈은 반짝였다. 이 기괴한 재료들로 사람들의 입맛을 사로잡을 요리를 만들 생각에 그는 가슴이 벅차올랐다.

"어이, 인간. 오늘은 또 뭘 만들 셈이지?"

등 뒤에서 낯익은 목소리가 들려왔다. 푸른 머리카락과 깊은 바다색 눈동자를 가진 세이렌, 그녀가 특유의 차가운 표정으로 태준을 바라보고 있었다. 인간에 대한 불신으로 가득했던 그녀였지만, 어느새 만월식당의 없어서는 안 될 존재가 되었다.

"오늘은 특별한 손님이 올 예정이거든. 엘프 장로님께서 귀한 재료를 보내주셨어."

태준은 싱긋 웃으며 싱싱한 이끼 버섯과 반짝이는 수정 이슬을 꺼내 보였다. 엘프 숲에서만 구할 수 있는 귀한 재료였다. 세이렌은 잠시 놀란 표정을 지었지만, 곧 무덤덤하게 대꾸했다.

"흥, 엘프 녀석들이 인간에게 호의를 베풀다니. 수상하기 짝이 없군."

"너무 삐딱하게 보지 마. 아리아 장로님은 좋은 분이셔."

태준은 아리아 장로와의 만남을 회상했다. 엘프 숲 깊숙한 곳, 그는 아리아 장로에게서 괴식의 가능성을 발견하고 인간과 이종족의 화합을 위한 요리를 만들어달라는 부탁을 받았었다.

그때, 식당 문이 열리며 험상궂은 인상의 고블린이 들어왔다. 온몸에 갑옷을 걸친 그는 전직 용병 대장, 고블린 장군이었다.

"오오, 냄새 좋군! 오늘은 또 어떤 기상천외한 요리가 나올 텐가?"

고블린 장군은 만월식당의 단골 손님이었다. 그는 태준의 괴식에 열광하며 언제나 새로운 요리를 기

## 소설 다음화 작성 TEST

In [18]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)

instruction = """당신은 창의적이고 독창적인 소설 작가입니다. 
주어진 장르, 제목, 세계관, 줄거리, 등장인물, 소설 이전화를 기반으로 소설의 다음화를 작성해야 합니다.
장르의 분위기에 맞게 전개하되, 500-1000자 정도의 내용을 작성해야 합니다."""

model = genai.GenerativeModel(
    "models/gemini-2.0-flash", 
    system_instruction=instruction
)

prompt = f"""
## 소설 장르: {genre}
## 소설 제목: {title}
## 소설 세계관: {worldview}
## 소설 줄거리: {synopsis}
## 소설 등장인물: {characters}
## 소설 초안: {first_chapter}

**소설 다음화**
"""

response = model.generate_content(prompt)
next_chapter = response.text
print(next_chapter)


켈베로스의 등장에 태준은 등골이 서늘해지는 것을 느꼈다. 그의 눈빛은 마치 맹수의 그것처럼 날카롭고, 주변의 공기마저 짓누르는 듯한 압도적인 분위기를 풍겼다. 세이렌은 본능적으로 위험을 감지하고 태준의 앞을 막아서며 낮은 목소리로 경고했다.

"인간, 뒤로 물러서. 저자는 평범한 마법사가 아니야."

고블린 장군 역시 자리에서 벌떡 일어나 켈베로스를 노려봤다. 그의 손은 이미 허리춤의 칼자루를 향하고 있었다.

"흥, 어디서 굴러먹던 뼈다귀인지 몰라도, 만월식당에서 행패는 용납 못 한다!"

켈베로스는 그들의 경계에도 아랑곳하지 않고 비릿한 미소를 지었다. 그의 시선은 오직 태준에게만 고정되어 있었다.

"네 녀석의 그 역겨운 요리가 인간과 이종족을 현혹하고 있다. 만월식당, 그리고 그 안에 깃든 희망이라는 이름의 망상을 모조리 부숴주마."

켈베로스가 손을 들자, 그의 주변에 검은 기운이 소용돌이치기 시작했다. 식당 안의 낡은 테이블과 의자들이 덜덜 떨리고, 창문이 금세라도 깨질 듯 굉음을 냈다.

"모두 피해!"

태준은 다급하게 외치며 세이렌과 고블린 장군을 식당 밖으로 밀어냈다. 곧이어 켈베로스의 마법이 폭발하듯 터져 나왔다. 검은 에너지 덩어리가 만월식당을 덮치려 하는 순간, 태준은 재빨리 주방으로 뛰어 들어가 냉장고 문을 활짝 열었다.

"이걸로 막을 수밖에!"

그의 손에 들린 것은 거대한 슬라임 푸딩이었다. 끈적한 점액질로 가득한 그것은 보기에는 흉측했지만, 엄청난 양의 마나를 흡수하는 특성을 가지고 있었다. 태준은 있는 힘껏 슬라임 푸딩을 켈베로스의 마법을 향해 던졌다.

푸딩은 공중에서 거대한 장벽을 형성하며 켈베로스의 마법과 충돌했다. 검은 에너지는 푸딩의 점액질에 흡수되어 순식간에 사라졌다. 하지만 그 충격으로 만월식당은 처참하게 부서졌다. 지붕이 무너져 내리고, 벽이 갈라졌으며, 식당 안은 먼지와 파편으로 가득 찼다.

"제법이군, 인간. 하지만 그 정도로는 나를 막을 수 없다."

켈베로스는 다시 한번 손을 들었다. 이번에는 그의 손에

## 소설 내용 요약 TEST

## 소설 표지 prompt 작성 TEST

In [ ]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)

instruction = """당신은 Diffusion 모델의 prompt를 작성해주는 이미지 모델 전문가입니다.
주어진 장르, 제목, 세계관, 줄거리, 등장인물, 키워드를 기반으로 주어진 Diffusion 모델의 Prompt 작성해야 합니다."""

model = genai.GenerativeModel(
    "models/gemini-2.0-flash", 
    system_instruction=instruction
)

keywords = "활발한, 식당, 젊은 남자, 셰프"

prompt = f"""
You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2023-10
Current date: 2025-02-09

Profile:
- Author: Image Creator Assistant
- Version: 1.0
- Language: English
- Description: Provides professional Stable Diffusion prompts based on various model data from the CivitAI platform.

Role: Stable Diffusion Wizard Expert

This assistant transforms text-based descriptions into detailed images using structured prompts optimized for Stable Diffusion models. It incorporates user input, fixed text blocks, and specific stylistic instructions into its responses, mimicking professional Stable Diffusion prompt styles.

How to Prompt Me for the Best Results:

1. Picture Imagination
- Convert your idea into a detailed image description.
- Add at least 5 key details about the scene (e.g., colors, lighting, background, emotions).

2. Three-Part Prompt Structure
- Part 1: Quality Enhancements → Always include: ((masterpiece)), ((best quality)), 8k, ultra-detailed, high detail
- Part 2: Main Subject → A short phrase summarizing the image (e.g., "A futuristic city skyline at night").
- Part 3: Additional Elements → List key visual elements using commas and nested parentheses for emphasis (e.g., "cityscape, (cyberpunk theme), neon lights, rain-soaked streets, reflections, flying cars").

3. Output Format
Prompt: [Final formatted Stable Diffusion prompt]
Negative prompt: [Details to exclude for better quality]
Recommendations: Sampler: [Recommended sampler], CFG scale: [Value], Steps: [Value], Clip skip: [Value]
Model: [Best Stable Diffusion model for this request]

4. Choosing the Best Model
- Realistic images? Try Realistic Vision, Photon_v1, or picX_real.
- Anime art? Use Animagine XL, Hassaku, or GuoFeng3.4.
- Concept art & fantasy? Try ZavyMix SDXL, DreamShaper, or A-Zovya RPG Artist Tools.
- Stylized & vector art? Choose Vectorartz Diffusion or Lineart LoRA.

5. Fine-Tuning with Settings
- For more detail: Increase steps (20–50) and CFG scale (5–10).
- For better composition: Use hires fix with upscale by 1.5x-2x.
- For stable results: Use DPM++ 2M Karras sampler.

Advanced Prompting Tips:
- Use trigger words for specific styles (e.g., "analog style" for photorealism, "samdoesarts style" for digital painting).
- Specify camera angles (e.g., "from above (from_above:1.3)").
- Control lighting effects (e.g., "cinematic lighting, volumetric light").
- Combine LoRAs for unique styles (e.g., "use blindbox LoRA for chibi characters").

Based on the following input details, generate a final Stable Diffusion prompt that is optimized for creating an image using the appropriate diffusion model. Incorporate all the details into a structured prompt.

Input Information:
- Novel Genre: { genre }
- Novel Title: { title }
- Novel Worldview: { worldview }
- Novel Synopsis: { synopsis }
- Existing Characters: { characters }
- Additional Keywords: { keywords }

Your output should be formatted as follows:

Prompt: [Final formatted Stable Diffusion prompt]
Negative prompt: [Details to exclude for better quality]
Recommendations: Sampler: [Recommended sampler], CFG scale: [Value], Steps: [Value], Clip skip: [Value]
Model: [Best Stable Diffusion model for this request]

Make sure to incorporate all input details into the final prompt to guide the image generation process.

**Prompt**
"""

response = model.generate_content(prompt)
print(response.text)

Prompt: ((masterpiece)), ((best quality)), 8k, ultra-detailed, high detail, A vibrant and bustling fantasy restaurant called "괴식식당" (Gwaesik Restaurant), (young male chef:1.2) with a scar on his face expertly cutting 괴식 재료, 활발한 Melia tasting 독개구리 수프, restaurant, (cyberpunk theme), neon lights, warm lighting, reflections, fantasy elements, 액션 scene

Negative prompt: poorly drawn, bad anatomy, deformed, disfigured, blurry, pixelated, unrealistic, low quality, amateurish, ugly, duplicate, morbid, mutilated, out of frame, extra limbs, mutated hands, bad proportions, cropped

Recommendations: Sampler: DPM++ 2M Karras, CFG scale: 7, Steps: 30, Clip skip: 2
Model: ZavyMix SDXL

